In [8]:
import pymysql
import os
from dotenv import load_dotenv
import requests
import json
import time

# Load environment variables
load_dotenv()

def get_connection():
    return pymysql.connect(
        host=os.getenv("DB_HOST"),
        port=int(os.getenv("DB_PORT")),
        user=os.getenv("DB_USER"),
        password=os.getenv("DB_PASSWORD"),
        database=os.getenv("DB_NAME"),
        cursorclass=pymysql.cursors.DictCursor
    )

def get_systems():
    connection = get_connection()
    cursor = connection.cursor()
    cursor.execute("SELECT id, topic FROM system_lists WHERE is_active = 1 ORDER BY topic ASC")
    systems = cursor.fetchall()
    cursor.close()
    connection.close()
    return systems

def get_questions(system_id, limit=1000):
    connection = get_connection()
    cursor = connection.cursor()
    query = '''
        SELECT q.id, q.already_updated, COALESCE(q.is_accepted, 0) AS is_accepted
        FROM questions_duplicated q
        JOIN subtopic_lists s ON q.subtopic_list_id = s.id
        JOIN topic_lists t ON s.topic_id = t.id
        WHERE t.system_id = %s
        ORDER BY q.id ASC
        LIMIT %s
    '''
    cursor.execute(query, (system_id, limit))
    questions = cursor.fetchall()
    cursor.close()
    connection.close()
    return questions

def get_question_detail(qid):
    connection = get_connection()
    cursor = connection.cursor()
    cursor.execute('''
        SELECT scenario, question, option_a, option_b, option_c, option_d, option_e, correct_answer, discussion, learning_objective
        FROM questions_duplicated WHERE id = %s
    ''', (qid,))
    question = cursor.fetchone()
    cursor.close()
    connection.close()
    return question

def update_question(qid, updated):
    connection = get_connection()
    cursor = connection.cursor()
    update_query = '''
        UPDATE questions_duplicated
        SET scenario=%s,
            question=%s,
            option_a=%s,
            option_b=%s,
            option_c=%s,
            option_d=%s,
            option_e=%s,
            correct_answer=%s,
            discussion=%s,
            learning_objective=%s,
            already_updated=1
        WHERE id=%s
    '''
    update_values = (
        updated['scenario'],
        updated['question'],
        updated['option_a'],
        updated['option_b'],
        updated['option_c'],
        updated['option_d'],
        updated['option_e'],
        updated['correct_answer'],
        updated['discussion'],
        updated['learning_objective'],
        qid
    )
    cursor.execute(update_query, update_values)
    connection.commit()
    cursor.close()
    connection.close()

def generate_new_question(original):
    prompt = f'''
Berikut ini adalah soal klinis tingkat UKMPPD yang telah ada, dalam format JSON dari aplikasi:

{original}

Tugas Anda:
1. **Identifikasi topik klinis atau diagnosis utama yang sedang diuji** **serta _jenis konsep_ yang ditanyakan** (mis. patofisiologi, diagnosis, tatalaksana, interpretasi EKG, radiologi, dll.) berdasarkan jawaban yang benar.
2. **Buat soal baru yang menguji topik _DAN_ jenis konsep yang sama**, namun dengan skenario klinis yang **sepenuhnya berbeda** (bukan parafrase, bukan pengulangan pola).
3. Skenario boleh berupa anamnesis, pemeriksaan fisik, EKG, hasil lab, radiologi, atau gabungan data klinis lainnya.
4. Jika pada soal asli terdapat gambar (misal: EKG, radiologi, foto lesi, grafik, dsb.) namun gambar tidak diberikan, gunakan clue dari soal/jawaban untuk menebak jenis gambar tersebut. Buat soal baru yang juga menggunakan gambar serupa (misal: deskripsikan hasil EKG, radiologi, dsb.) dan pastikan soal baru tetap relevan dengan konteks gambar tersebut. Tidak perlu menjelaskan proses penebakan gambar pada output—cukup hasil akhirnya saja.
5. Jika jenis konsep yang diidentifikasi **bukan “diagnosis”**, maka fokus soal dan opsi jawaban harus tetap pada konsep tersebut (contoh: mekanisme patofisiologi, pilihan obat, interpretasi grafis, dsb.).  
   Jika memang “diagnosis”, barulah fokus soal pada penegakan diagnosis.
6. Buat lima pilihan jawaban (A–E) dengan tepat satu yang benar; pastikan semua opsi kredibel secara medis.
7. Buat **pembahasan komprehensif**, menjelaskan:  
   - Mengapa jawaban benar paling tepat.  
   - Mengapa masing‑masing opsi lain salah, dikaitkan dengan data klinis.
8. **Tentukan “learning_objective”**—konsep inti yang harus dikuasai calon dokter untuk menjawab soal tersebut (≤ 40 kata; ringkas, langsung ke inti; tidak perlu mengungkap detail skenario klinis).

Outputkan **hanya satu blok JSON** dengan struktur:
{{
  "scenario": "",
  "question": "",
  "option_a": "",
  "option_b": "",
  "option_c": "",
  "option_d": "",
  "option_e": "",
  "correct_answer": "",
  "discussion": "",
  "learning_objective": ""
}}

Gunakan Bahasa Indonesia akademik yang jelas, logis, dan mengalir klinis.
'''
    api_key = os.getenv('GROK_API_KEY')
    headers = {'Authorization': f'Bearer {api_key}'}
    payload = {
        'model': 'grok-3',
        'messages': [
            {
                'role': 'system',
                'content': 'You are an expert medical question generator for a clinical education app. Your job is to create new, high-quality clinical case questions (with scenario, options, correct answer, discussion, and learning objective) for Indonesian medical students, following strict academic and clinical standards.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ],
        'max_tokens': 4000,
        'temperature': 0.9,
        'stream': False
    }
    response = requests.post('https://api.x.ai/v1/chat/completions', json=payload, headers=headers)
    result = response.json()
    # Try to extract the JSON result
    if "choices" in result and result["choices"]:
        content = result["choices"][0]["message"]["content"]
        return json.loads(content)
    elif all(k in result for k in ["scenario","question","option_a","option_b","option_c","option_d","option_e","correct_answer","discussion","learning_objective"]):
        return result
    elif "result" in result and isinstance(result["result"], dict):
        return result["result"]
    else:
        raise Exception(f"Unknown Grok response: {result}")

# --- Main notebook logic ---

systems = get_systems()
print("Available systems:")
for s in systems:
    print(f"{s['id']}: {s['topic']}")
system_id = int(input("Enter the ID of the system to update: "))

questions = get_questions(system_id)
to_update = [q for q in questions if not q['already_updated']]

print(f"Found {len(to_update)} questions to update.")

for q in to_update:
    print(f"Updating question ID {q['id']} ...")
    original = get_question_detail(q['id'])
    if not original:
        print(f"Question {q['id']} not found, skipping.")
        continue
    try:
        updated = generate_new_question(original)
        update_question(q['id'], updated)
        print(f"Question {q['id']} updated.")
    except Exception as err:
        print(f"Failed to update question {q['id']}: {err}")
    time.sleep(10)  # Avoid rate limits

print("All questions updated for this system.")

Available systems:
7: Dermatovenerologi
13: Endokrin-Gizi-Metabolik
12: Forensik
5: Gastroenterohepatologi
10: Ginjal-Saluran-Kemih
9: Hematoimunologi
8: Kardiovaskular
15: Lainnya
6: Mata
11: Muskuloskeletal
2: Psikiatri
14: Reproduksi
3: Respirasi
1: Saraf
4: THT-KL
Enter the ID of the system to update: 3
Found 69 questions to update.
Updating question ID 955 ...
Question 955 updated.
Updating question ID 956 ...
Question 956 updated.
Updating question ID 957 ...
Question 957 updated.
Updating question ID 958 ...
Question 958 updated.
Updating question ID 959 ...
Question 959 updated.
Updating question ID 960 ...
Failed to update question 960: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spending limit.'}
Updating question ID 961 ...
Failed to update ques

Updating question ID 982 ...
Failed to update question 982: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spending limit.'}
Updating question ID 983 ...
Failed to update question 983: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spending limit.'}
Updating question ID 984 ...
Failed to update question 984: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API req

Updating question ID 1005 ...
Failed to update question 1005: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spending limit.'}
Updating question ID 1006 ...
Failed to update question 1006: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spending limit.'}
Updating question ID 1007 ...
Failed to update question 1007: Unknown Grok response: {'code': 'Some resource has been exhausted', 'error': 'Your team 2fc00ca1-e921-4231-8005-0c7bed34707c has either used all available credits or reached its monthly spending limit. To continue making A